In [1]:
#task h3.1: Сделать свое решение для этого соревнования https://www.kaggle.com/competitions/digit-recognizer
# Результат: опубликовать решение в разделе. После отправить мне через сайт ку решение и какой у вас получился скор

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [3]:
train = pd.read_csv('digit-recognizer/train.csv')
test = pd.read_csv('digit-recognizer/test.csv')

In [4]:
y_train = train["label"]
X_train = train.drop(labels = ["label"], axis=1)

In [5]:
X_train = X_train / 255.0
test = test / 255.0

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [6]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1
)
datagen.fit(X_train)

In [8]:
model = Sequential()

# Первый сверточный блок
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='Same', input_shape=(28,28,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='Same', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='Same'))
model.add(MaxPooling2D(pool_size=(2,2)))


# Переход к полносвязным слоям
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Компиляция модели
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
epochs = 10
batch_size = 32

In [11]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


242/242 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.6021 - loss: 1.1550 - val_accuracy: 0.9640 - val_loss: 0.1157
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9287 - loss: 0.2283 - val_accuracy: 0.9721 - val_loss: 0.0906
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9526 - loss: 0.1602 - val_accuracy: 0.9767 - val_loss: 0.0795
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9593 - loss: 0.1448 - val_accuracy: 0.9860 - val_loss: 0.0597
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9670 - loss: 0.1131 - val_accuracy: 0.9814 - val_loss: 0.0651
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9615 - loss: 0.1229 - val_accuracy: 0.9837 - val_loss: 0.0719
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9711 - loss: 0.0911 - val_accuracy: 0.9849 - val_loss: 0.0701
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9759 - loss: 0.0771 - val_accuracy: 0.99

In [12]:
pred = model.predict(test, verbose=1)

234/234 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [13]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [14]:
# Save the final result in cnn_mnist_submission.csv
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_submission.csv",index=False)